In [2]:
# -*- coding: utf-8 -*-
import os
import pandas as pd
import numpy as np
from PIL import Image
import gc, math
import pickle

from keras.models import Sequential
from keras.optimizers import SGD
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.models import model_from_json
from keras.models import Model
from keras.layers import Input, Dense, Convolution2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D, Dropout, Flatten, merge, Reshape, Activation
from keras.layers.advanced_activations import LeakyReLU, PReLU
from keras.layers.normalization import BatchNormalization
from keras import regularizers
from keras import backend as K
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import log_loss, accuracy_score, confusion_matrix

from cnnmodels import vgg_std16_model, preprocess_input, create_rect5, load_img, train_generator, test_generator

In [4]:
# Params
img_rows, img_cols = 224, 224 # Resolution of inputs
channel = 3
num_class = 2
ROWS, COLS = 224, 224
BATCHSIZE = 128
SEAL_CLASSES = ['NoS', 'seal']
nb_perClass = int(BATCHSIZE / len(SEAL_CLASSES))
TRAIN_DIR = '../darknet/seals/JPEGImagesBlk'
TEST_DIR = '../darknet/seals/JPEGImagesTest'
# Data generator
train_datagen = ImageDataGenerator(
    horizontal_flip=True,
    vertical_flip=True)
# Fine-tune Example
rfcnCV = pd.read_pickle('../coords/rfcnCV.pkl')
rfcnCV.head(3)

In [10]:
# Lets make our validation set
folds = [rfcnCV.img.str.split('_').apply(lambda x: x[0]).astype(int) % 2 != 0,
        rfcnCV.img.str.split('_').apply(lambda x: x[0]).astype(int) % 2 == 0]

for fold in range(2):
    train_df = rfcnCV[~folds[fold]]
    valid_df = rfcnCV[folds[fold]]

    # validation_data (valid_x,valid_y)
    df_1 = valid_df
    l = valid_df.groupby('seal').size()
    nb_NoF_valid = math.ceil(l.sum()/10)
    valid_x = np.zeros((valid_df.shape[0], ROWS, COLS, 3), dtype=K.floatx())
    valid_y = np.zeros((valid_df.shape[0], len(SEAL_CLASSES)), dtype=K.floatx())
    i = 0
    for index,row in valid_df.iterrows():
        row = row.tolist()
        image_file = os.path.join(TRAIN_DIR, row[0])
        seal = row[6]
        bbox = row[2:6]
        cropped = load_img(image_file+'.jpg',bbox,target_size=(ROWS,COLS))
        x = np.asarray(cropped, dtype=K.floatx())
        x = preprocess_input(x)
        valid_x[i] = x
        valid_y[i,seal] = 1
        i += 1
    valid_x = valid_x.transpose(0, 3, 1, 2)
    valid_x.shape

    # Load our model
    nb_epoch = 2
    samples_per_epoch = 50000
    model = vgg_std16_model(ROWS, COLS, channel, num_class)

    # Start Fine-tuning
    model.fit_generator(train_generator(train_datagen, train_df),
              nb_epoch=nb_epoch,
              samples_per_epoch=samples_per_epoch, #50000,
              verbose=1,
              validation_data=(valid_x, valid_y),
              )

    for layer in model.layers[10:]:
        layer.trainable = True
    model.optimizer.lr = 1e-4
    nb_epoch = 3
    model.fit_generator(train_generator(train_datagen, df=train_df),
              nb_epoch=nb_epoch,
              samples_per_epoch=samples_per_epoch,
              verbose=1,
              validation_data=(valid_x, valid_y),
              )

    # Test preds save
    test_preds = model.predict_generator(test_generator(valid_df), val_samples=valid_df.shape[0])
    df = pd.concat([valid_df, pd.DataFrame(test_preds,  columns=['predNoSeal', 'predSeal'])], axis=1)
    df.to_pickle('../coords/vggCVPreds2604_fold' + str(fold+1) + '.pkl')
    df[['img', 'predSeal']].to_csv('../coords/vggCVPreds2604_fold' + str(fold+1) + '.csv', index=False)

Epoch 1/3
70016/70000 [==============================] - 4190s - loss: 0.0899 - acc: 0.9680 - val_loss: 0.0851 - val_acc: 0.9683
Epoch 2/3
70016/70000 [==============================] - 4186s - loss: 0.0818 - acc: 0.9714 - val_loss: 0.0812 - val_acc: 0.9701
Epoch 3/3
70016/70000 [==============================] - 4184s - loss: 0.0725 - acc: 0.9733 - val_loss: 0.0748 - val_acc: 0.9729
